In [24]:
r_global = 0.1
k_global = 0
test_global = 0.1
test_global_num = None
val_global = 0.1
val_global_num = None
dataset_global = "Cora"
thresh = 1e-10
alpha_param,beta_param,gamma_param,lambda_param,delta_param = 100.0,1.0,100.0,0.01,1e-06,

In [25]:
from datetime import datetime

from networkx.algorithms import community
from networkx.generators.community import random_partition_graph
from networkx.generators.community import stochastic_block_model
from networkx.generators.random_graphs import barabasi_albert_graph
from networkx.generators.random_graphs import erdos_renyi_graph
from networkx.generators.random_graphs import watts_strogatz_graph
from random import sample
from scipy.sparse import csgraph
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import random
from scipy.sparse.linalg import inv
from scipy.sparse.linalg import norm
from scipy.stats import rv_continuous
from sklearn.decomposition import FactorAnalysis
from torch import Tensor
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.nn import GCNConv
from torch_geometric.utils import dense_to_sparse,homophily
from torch_geometric.utils import to_dense_adj
from torch_geometric.utils import to_networkx
from tqdm import tqdm
import collections
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import pandas as pd
from pyswarm import pso
import scipy.sparse as sp
import seaborn as sns
import torch.nn.functional as F
import torch_geometric
import torch

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)
print("Checking if data exists. If not downloading the data.")
dataset = Planetoid(root='data/planetoid', name=dataset_global)
print("Dataset Information")
print(dataset[0])
dataset = dataset[0]
print("-------------||-------------")
print("Number of Nodes :",dataset.x.shape[0])
print("The data will be split into training, tesing and validation")
test_global_num = int(dataset.x.shape[0]*(test_global))
val_global_num = int(dataset.x.shape[0]*(val_global))
print("Ratio of Training:",(1-test_global-val_global),"No of Nodes =",dataset.x.shape[0]-test_global_num-val_global_num)
print("Ratio of Testing:",(test_global),", No of edges =",test_global_num)
print("Ratio of Validation:",(val_global),", No of Nodes =",val_global_num)
print("-------------||-------------")


Checking if data exists. If not downloading the data.
Dataset Information
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
-------------||-------------
Number of Nodes : 2708
The data will be split into training, tesing and validation
Ratio of Training: 0.8 No of Nodes = 2168
Ratio of Testing: 0.1 , No of edges = 270
Ratio of Validation: 0.1 , No of Nodes = 270
-------------||-------------


In [27]:
print("-------------||-------------")
print("Restructuring the database into testing, traing, validation")
dataset = Planetoid(root='data/planetoid', split="random",num_val= val_global_num, num_test= test_global_num, name=dataset_global)
dataset = dataset[0]
print("Dataset Information:")
print(dataset)
print("Number of Nodes :",dataset.x.shape[0])
NO_OF_CLASSES = len(set(dataset.y.numpy()))
print("Number of classes:",NO_OF_CLASSES)
print()
print("The data will be split into training, tesing and validation")
test_global_num = int(dataset.x.shape[0]*(test_global))
val_global_num = int(dataset.x.shape[0]*(val_global))
train_global_num = dataset.x.shape[0]-test_global_num-val_global_num
print("Ratio of Training:",(1-test_global-val_global),"No of Nodes =",dataset.x.shape[0]-test_global_num-val_global_num)
print("Ratio of Testing:",(test_global),", No of edges =",test_global_num)
print("Ratio of Validation:",(val_global),", No of Nodes =",val_global_num)
print()
print("Homophilic ratio : " + str(homophily(dataset.edge_index,dataset.y,method='edge')))
NO_OF_EDGES = dataset.edge_index.shape[1]
N = dataset.x.shape[0]
sparsity = 2*NO_OF_EDGES/(N*(N-1))
print("Sparsity of Graph:",sparsity)
print("-------------||-------------")


-------------||-------------
Restructuring the database into testing, traing, validation
Dataset Information:
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of Nodes : 2708
Number of classes: 7

The data will be split into training, tesing and validation
Ratio of Training: 0.8 No of Nodes = 2168
Ratio of Testing: 0.1 , No of edges = 270
Ratio of Validation: 0.1 , No of Nodes = 270

Homophilic ratio : 0.8099659085273743
Sparsity of Graph: 0.0028799998253884154
-------------||-------------


In [28]:
print("-------------||-------------")
print("Generate Adjecency Matrix")
adj = to_dense_adj(dataset.edge_index)
adj = adj[0]
print("Adjacency Matrix:", adj.shape)
print(adj)
print("\nConvert Adjacency to Laplacian Matrix\n")
def get_laplacian(adj):
    b=torch.ones(adj.shape[0])
    return torch.diag(adj@b)-adj
theta = get_laplacian(adj)
print("Laplacian Matrix:", theta.shape)
print(theta)
print("-------------||-------------")


-------------||-------------
Generate Adjecency Matrix
Adjacency Matrix: torch.Size([2708, 2708])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.]])

Convert Adjacency to Laplacian Matrix

Laplacian Matrix: torch.Size([2708, 2708])
tensor([[ 3.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  3., -1.,  ...,  0.,  0.,  0.],
        [ 0., -1.,  5.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  4., -1.],
        [ 0.,  0.,  0.,  ...,  0., -1.,  4.]])
-------------||-------------


In [29]:
print("-------------||-------------")
print("Splitting Adjacency Matrix into Training, Testing, Validation")
print("Training:")
TRAIN_adj = adj[:dataset.x.shape[0]-test_global_num-val_global_num,:dataset.x.shape[0]-test_global_num-val_global_num]
print(TRAIN_adj.shape)
print(TRAIN_adj)

print("Testing:")
TEST_adj = adj[train_global_num:train_global_num+test_global_num,train_global_num:train_global_num+test_global_num]
print(TEST_adj.shape)
print(TEST_adj)

print("Validation:")
VAL_adj = adj[train_global_num+test_global_num:train_global_num+test_global_num+val_global_num,train_global_num+test_global_num:train_global_num+test_global_num+val_global_num]
print(VAL_adj.shape)
print(VAL_adj)
print("-------------||-------------")


-------------||-------------
Splitting Adjacency Matrix into Training, Testing, Validation
Training:
torch.Size([2168, 2168])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Testing:
torch.Size([270, 270])
tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Validation:
torch.Size([270, 270])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.]])
-------------||-------------


In [30]:
print("-------------||-------------")
print("Splitting Laplacian Matrix into Training, Testing, Validation")
print("Training:")
TRAIN_theta = adj[:dataset.x.shape[0]-test_global_num-val_global_num,:dataset.x.shape[0]-test_global_num-val_global_num]
TRAIN_theta = get_laplacian(TRAIN_theta)
print(TRAIN_theta.shape)
print(TRAIN_theta)

print("Testing:")
TEST_theta = adj[train_global_num:train_global_num+test_global_num,train_global_num:train_global_num+test_global_num]
TEST_theta = get_laplacian(TEST_theta)
print(TEST_theta.shape)
print(TEST_theta)

print("Validation:")
VAL_theta = adj[train_global_num+test_global_num:train_global_num+test_global_num+val_global_num,train_global_num+test_global_num:train_global_num+test_global_num+val_global_num]
VAL_theta = get_laplacian(VAL_theta)
print(VAL_theta.shape)
print(VAL_theta)
print("-------------||-------------")


-------------||-------------
Splitting Laplacian Matrix into Training, Testing, Validation
Training:
torch.Size([2168, 2168])
tensor([[ 2.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  3., -1.,  ...,  0.,  0.,  0.],
        [ 0., -1.,  5.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  3.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  2.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  1.]])
Testing:
torch.Size([270, 270])
tensor([[ 1., -1.,  0.,  ...,  0.,  0.,  0.],
        [-1.,  1.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]])
Validation:
torch.Size([270, 270])
tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  2.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ..., 

In [31]:
print("-------------||-------------")
print("Generating Edge Indexes for Traing, Testing and Validation")
def adj2edge_index(adj):
    edge_index = np.vstack(np.nonzero(adj)).T
    return torch.from_numpy(edge_index)
TRAIN_edge_index = adj2edge_index(TRAIN_adj)
print(TRAIN_edge_index)
print(TRAIN_edge_index.shape)

TEST_edge_index = adj2edge_index(TEST_adj)
print(TEST_edge_index)
print(TEST_edge_index.shape)

VAL_edge_index = adj2edge_index(VAL_adj)
print(VAL_edge_index)
print(VAL_edge_index.shape)
print("-------------||-------------")

-------------||-------------
Generating Edge Indexes for Traing, Testing and Validation
tensor([[   0,    0,    1,  ..., 2166, 2166, 2167],
        [ 633, 1862,    2,  ...,  118, 1029,  957]])
torch.Size([2, 7754])
tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,   9,  12,  12,  13,
          13,  13,  14,  14,  14,  14,  14,  14,  15,  18,  25,  26,  26,  27,
          28,  29,  30,  31,  31,  31,  32,  33,  34,  35,  39,  43,  44,  46,
          47,  48,  48,  48,  52,  53,  54,  54,  55,  56,  56,  56,  56,  57,
          57,  58,  59,  60,  60,  60,  62,  63,  63,  64,  65,  65,  68,  69,
          69,  70,  70,  70,  71,  72,  72,  73,  73,  74,  75,  75,  75,  76,
          77,  77,  78,  78,  78,  79,  80,  81,  83,  84,  91,  91,  98,  98,
          98,  99, 100, 101, 102, 102, 113, 116, 117, 117, 118, 119, 120, 123,
         123, 123, 124, 124, 128, 129, 129, 130, 131, 136, 137, 138, 153, 159,
         160, 161, 161, 162, 162, 163, 167, 167, 168, 169, 171, 172, 175, 

In [32]:
print("-------------||-------------")
print("Defining Feature Matrix for Training, Testing, Validation")
TRAIN_X = dataset.x[:train_global_num]
print("TRAIN_X Shape:",TRAIN_X.shape)
print(TRAIN_X)
TEST_X = dataset.x[train_global_num:train_global_num+test_global_num]
print("TEST_X Shape:",TEST_X.shape)
print(TEST_X)
VAL_X = dataset.x[train_global_num+test_global_num:train_global_num+test_global_num+val_global_num]
print("VAL_X Shape:",VAL_X.shape)
print(VAL_X)
print("-------------||-------------")

-------------||-------------
Defining Feature Matrix for Training, Testing, Validation
TRAIN_X Shape: torch.Size([2168, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
TEST_X Shape: torch.Size([270, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
VAL_X Shape: torch.Size([270, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
-------------||-------------


In [33]:
print("-------------||-------------")
print("Defining Label Matrix for Training, Testing, Validation")
TRAIN_Y = dataset.y[:train_global_num]
print("TRAIN_Y Shape:",TRAIN_Y.shape)
print(TRAIN_Y)
TEST_Y = dataset.y[train_global_num:train_global_num+test_global_num]
print("TEST_Y Shape:",TEST_Y.shape)
print(TEST_Y)
VAL_Y = dataset.y[train_global_num+test_global_num:train_global_num+test_global_num+val_global_num]
print("VAL_Y Shape:",VAL_Y.shape)
print(VAL_Y)
print("-------------||-------------")

-------------||-------------
Defining Label Matrix for Training, Testing, Validation
TRAIN_Y Shape: torch.Size([2168])
tensor([3, 4, 4,  ..., 6, 5, 2])
TEST_Y Shape: torch.Size([270])
tensor([1, 2, 1, 0, 0, 6, 6, 2, 3, 3, 5, 0, 0, 0, 0, 0, 5, 5, 0, 3, 5, 0, 6, 3,
        6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 1, 6, 1, 0, 3, 3, 3, 3,
        3, 6, 1, 0, 2, 2, 4, 4, 4, 4, 4, 5, 6, 3, 3, 0, 0, 0, 0, 5, 4, 4, 4, 4,
        4, 3, 3, 3, 3, 3, 0, 3, 4, 4, 4, 1, 1, 3, 1, 1, 5, 1, 3, 4, 4, 4, 4, 4,
        4, 4, 0, 0, 0, 5, 5, 5, 5, 5, 0, 5, 3, 0, 6, 2, 0, 5, 3, 3, 5, 5, 5, 5,
        5, 4, 4, 0, 4, 0, 4, 0, 3, 4, 4, 4, 1, 3, 3, 3, 3, 3, 4, 2, 3, 3, 3, 0,
        0, 2, 3, 3, 3, 3, 1, 1, 3, 0, 1, 4, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 2, 4,
        4, 4, 3, 3, 3, 4, 0, 3, 3, 3, 3, 0, 3, 3, 4, 4, 4, 4, 4, 4, 0, 4, 3, 2,
        0, 3, 4, 5, 0, 2, 2, 3, 3, 3, 3, 3, 2, 3, 5, 5, 4, 1, 4, 4, 4, 3, 4, 4,
        0, 4, 4, 4, 5, 2, 2, 2, 2, 4, 6, 6, 6, 6, 3, 4, 4, 4, 1, 3, 0, 3, 3, 5,
        0, 2, 3,

In [34]:
print("-------------||-------------")
print("Defining P,K,n for Training, Testing, Validation")
TRAIN_P = TRAIN_X.shape[0]
TEST_P = TEST_X.shape[0]
VAL_P = VAL_X.shape[0]
TRAIN_K = int(TRAIN_P*r_global)
TEST_K  = int(TEST_P*r_global)
VAL_K = int(VAL_P*r_global)
TRAIN_n = TRAIN_X.shape[1]
TEST_n = TEST_X.shape[1]
VAL_n = VAL_X.shape[1]
print("0","\t","TRAIN","\t","TEST","\t","VAL")
print("P","\t",TRAIN_P,"\t",TEST_P,"\t",VAL_P)
print("K","\t",TRAIN_K,"\t",TEST_K,"\t",VAL_K)
print("n","\t",TRAIN_n,"\t",TEST_n,"\t",VAL_n)
print("-------------||-------------")


-------------||-------------
Defining P,K,n for Training, Testing, Validation
0 	 TRAIN 	 TEST 	 VAL
P 	 2168 	 270 	 270
K 	 216 	 27 	 27
n 	 1433 	 1433 	 1433
-------------||-------------


In [35]:
print("-------------||-------------")
print("Generating CustomDistributions for Training, Testing, Validation")
class CustomDistribution(rv_continuous):
    def _rvs(self,  size=None, random_state=None):
        return random_state.standard_normal(size)
print("Generating for Training")
TRAIN_temp = CustomDistribution(seed=1)
TRAIN_temp2 = TRAIN_temp()  # get a frozen version of the distribution
TRAIN_X_tilde = random(TRAIN_K, TRAIN_n, density=0.25, random_state=1, data_rvs=TRAIN_temp2.rvs)
TRAIN_C = random(TRAIN_P, TRAIN_K, density=0.25, random_state=1, data_rvs=TRAIN_temp2.rvs)
print("Generating for Testing")
TEST_temp = CustomDistribution(seed=1)
TEST_temp2 = TEST_temp()  # get a frozen version of the distribution
TEST_X_tilde = random(TEST_K, TEST_n, density=0.25, random_state=1, data_rvs=TEST_temp2.rvs)
TEST_C = random(TEST_P, TEST_K, density=0.25, random_state=1, data_rvs=TEST_temp2.rvs)
print("Generating for Validation")
VAL_temp = CustomDistribution(seed=1)
VAL_temp2 = TEST_temp()  # get a frozen version of the distribution
VAL_X_tilde = random(VAL_K, VAL_n, density=0.25, random_state=1, data_rvs=VAL_temp2.rvs)
VAL_C = random(VAL_P, VAL_K, density=0.25, random_state=1, data_rvs=VAL_temp2.rvs)
print("-------------||-------------")


-------------||-------------
Generating CustomDistributions for Training, Testing, Validation
Generating for Training
Generating for Testing
Generating for Validation
-------------||-------------


In [36]:
print("-------------||-------------")
print("Additional Variabled Required: Labels")
TRAIN_labels = TRAIN_Y
TRAIN_labels = TRAIN_labels.numpy()
print("TRAIN_Labels\n",TRAIN_labels)
TEST_labels = TEST_Y
TEST_labels = TEST_labels.numpy()
print("TEST_Labels\n",TEST_labels)
VAL_labels = VAL_Y
VAL_labels = VAL_labels.numpy()
print("VAL_Labels\n",VAL_labels)
print()
print("Additional Variabled Required: Features")
TRAIN_features = TRAIN_X.numpy()
TEST_features = TEST_X.numpy()
VAL_features = VAL_X.numpy()
print("TRAIN_features\n",TRAIN_features)
print("TEST_features\n",TEST_features)
print("VAL_features\n",VAL_features)



-------------||-------------
Additional Variabled Required: Labels
TRAIN_Labels
 [3 4 4 ... 6 5 2]
TEST_Labels
 [1 2 1 0 0 6 6 2 3 3 5 0 0 0 0 0 5 5 0 3 5 0 6 3 6 0 0 0 0 0 0 0 0 0 0 0 3
 3 3 3 1 6 1 0 3 3 3 3 3 6 1 0 2 2 4 4 4 4 4 5 6 3 3 0 0 0 0 5 4 4 4 4 4 3
 3 3 3 3 0 3 4 4 4 1 1 3 1 1 5 1 3 4 4 4 4 4 4 4 0 0 0 5 5 5 5 5 0 5 3 0 6
 2 0 5 3 3 5 5 5 5 5 4 4 0 4 0 4 0 3 4 4 4 1 3 3 3 3 3 4 2 3 3 3 0 0 2 3 3
 3 3 1 1 3 0 1 4 1 1 1 1 1 1 0 1 0 0 2 4 4 4 3 3 3 4 0 3 3 3 3 0 3 3 4 4 4
 4 4 4 0 4 3 2 0 3 4 5 0 2 2 3 3 3 3 3 2 3 5 5 4 1 4 4 4 3 4 4 0 4 4 4 5 2
 2 2 2 4 6 6 6 6 3 4 4 4 1 3 0 3 3 5 0 2 3 3 3 3 3 2 4 4 0 0 3 2 6 6 0 3 3
 3 5 1 3 4 4 2 4 4 4 3]
VAL_Labels
 [3 2 2 2 2 2 2 2 2 2 2 0 2 2 2 0 6 6 5 6 6 3 2 6 3 4 4 4 2 6 6 0 0 3 0 4 4
 3 2 3 1 6 6 5 3 4 3 5 3 1 1 3 4 5 2 3 3 3 4 5 4 0 3 3 0 2 1 1 5 2 3 3 5 0
 2 3 2 2 5 5 4 3 4 3 2 2 4 2 4 5 5 3 2 3 1 0 3 3 4 5 4 3 3 3 3 3 0 1 2 4 4
 4 3 3 3 5 2 3 2 2 2 3 2 2 0 4 4 3 3 3 3 3 3 3 3 3 3 0 0 3 0 3 0 2 3 4 1 2
 5 4 3 3 3 1 5 3 4 3 2 2 1 

In [37]:
def convertScipyToTensor(coo):
    try:
        coo = coo.tocoo()
    except:
        coo = coo
    values = coo.data
    indices = np.vstack((coo.row, coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))

In [38]:
def experiment(alpha_param,beta_param,gamma_param,lambda_param,delta_param,C,X_tilde,theta,X,Y):
    p = X.shape[0]
    k = int(p*r_global)
    n = X.shape[1]
    ones = csr_matrix(np.ones((k,k)))
    ones = convertScipyToTensor(ones)
    ones = ones.to_dense()
    J = np.outer(np.ones(k), np.ones(k))/k
    J = csr_matrix(J)
    J = convertScipyToTensor(J)
    J = J.to_dense()
    zeros = csr_matrix(np.zeros((p,k)))
    zeros = convertScipyToTensor(zeros)
    zeros = zeros.to_dense()
    X_tilde = convertScipyToTensor(X_tilde)
    X_tilde = X_tilde.to_dense()
    C = convertScipyToTensor(C)
    C = C.to_dense()
    eye = torch.eye(k)
    try:
        theta = convertScipyToTensor(theta)
    except:
        theta = theta
    try:
        X = convertScipyToTensor(X)
        X = X.to_dense()
    except:
        X = X

    def one_hot(x, class_count):
        return torch.eye(class_count)[x, :]

    P = Y.numpy()
    NO_OF_CLASSES = len(set(Y))
    P = one_hot(P,NO_OF_CLASSES)
    
    if(torch.cuda.is_available()):
        # print("yes")
        X_tilde = X_tilde.cuda()
        C = C.cuda()
        theta = theta.cuda()
        X = X.cuda()
        J = J.cuda()
        P = P.cuda()
        zeros = zeros.cuda()
        ones = ones.cuda()
        eye = eye.cuda()
    def update(X_tilde,C,i):
        global L
        thetaC = theta@C
        CT = torch.transpose(C,0,1)
        X_tildeT = torch.transpose(X_tilde,0,1)
        CX_tilde = C@X_tilde
        t1 = CT@thetaC + J
        term_bracket = torch.linalg.pinv(t1)
        thetacX_tilde = thetaC@(X_tilde)
        
        L = 1/k

        t1 = -2*gamma_param*(thetaC@term_bracket)
        t2 = alpha_param*(CX_tilde-X)@(X_tildeT)
        t3 = 2*thetacX_tilde@(X_tildeT)
        t4 = lambda_param*(C@ones)
        t5 = 2*beta_param*(thetaC@CT@thetaC)
        t6 = delta_param*P@torch.transpose((CT@P),0,1)
        T2 = (t1+t2+t3+t4+t5+t6)/L
        Cnew = (C-T2).maximum(zeros)
        t1 = CT@thetaC*(2/alpha_param)
        t2 = CT@C
        t1 = torch.linalg.pinv(t1+t2)
        t1 = t1@CT
        t1 = t1@X
        X_tilde_new = t1
        Cnew[Cnew<thresh] = thresh
        for i in range(len(Cnew)):
            Cnew[i] = Cnew[i]/torch.linalg.norm(Cnew[i],1)
        for i in range(len(X_tilde_new)):
            X_tilde_new[i] = X_tilde_new[i]/torch.linalg.norm(X_tilde_new[i],1)
        return X_tilde_new,Cnew


    for i in tqdm(range(20)):
        X_tilde,C = update(X_tilde,C,i)

    return X_tilde,C



In [39]:
print("-------------||-------------")
print("Generating X_t_0,C_0")
print("-------------|TRAINING|-------------")
TRAIN_X_t_0,TRAIN_C_0 = experiment(alpha_param,beta_param,gamma_param,lambda_param,delta_param,TRAIN_C,TRAIN_X_tilde,TRAIN_theta,TRAIN_X,TRAIN_Y)
TRAIN_L = TRAIN_theta
TRAIN_C_0 = TRAIN_C_0.cpu().detach().numpy()
TRAIN_X_t_0 = TRAIN_X_t_0.cpu().detach().numpy()
TRAIN_C_t_0 = TRAIN_C_0.T
try:
    TRAIN_L = TRAIN_L.cpu().detach().numpy()
except:
    TRAIN_L = TRAIN_L
print("-------------|TESTING|-------------")
TEST_X_t_0,TEST_C_0 = experiment(alpha_param,beta_param,gamma_param,lambda_param,delta_param,TEST_C,TEST_X_tilde,TEST_theta,TEST_X,TEST_Y)
TEST_L = TEST_theta
TEST_C_0 = TEST_C_0.cpu().detach().numpy()
TEST_X_t_0 = TEST_X_t_0.cpu().detach().numpy()
TEST_C_t_0 = TEST_C_0.T
try:
    TEST_L = TEST_L.cpu().detach().numpy()
except:
    TEST_L = TEST_L
print("-------------|VALIDATION|-------------")
VAL_X_t_0,VAL_C_0 = experiment(alpha_param,beta_param,gamma_param,lambda_param,delta_param,VAL_C,VAL_X_tilde,VAL_theta,VAL_X,VAL_Y)
VAL_L = VAL_theta
VAL_C_0 = VAL_C_0.cpu().detach().numpy()
VAL_X_t_0 = VAL_X_t_0.cpu().detach().numpy()
VAL_C_t_0 = VAL_C_0.T
try:
    VAL_L = VAL_L.cpu().detach().numpy()
except:
    VAL_L = VAL_L

print("-------------||-------------")

-------------||-------------
Generating X_t_0,C_0
-------------|TRAINING|-------------


100%|██████████| 20/20 [00:05<00:00,  3.38it/s]


-------------|TESTING|-------------


100%|██████████| 20/20 [00:00<00:00, 64.66it/s]


-------------|VALIDATION|-------------


100%|██████████| 20/20 [00:00<00:00, 69.50it/s]

-------------||-------------


In [40]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(TRAIN_X.shape[1], 64)
        self.conv2 = GCNConv(64, NO_OF_CLASSES)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, TRAIN_x, edge_index):

        #print("Checking 1: x", x.shape, "Edge index:", edge_index.shape)
        TRAIN_x = self.conv1(TRAIN_x, edge_index)
        #print("Checking 2: convolution done, new x:", x.shape)
        TRAIN_x = F.relu(TRAIN_x)
        #print("Checking 3: x", x.shape, "training:", self.training)
        TRAIN_x = F.dropout(TRAIN_x, training=self.training)
        #print("Checking 4: dropout done new x", x.shape, "Edge index:", edge_index.shape)
        TRAIN_x = self.conv2(TRAIN_x, edge_index)
        #print("Checking 5: x", x.shape)

        return F.log_softmax(TRAIN_x, dim=1)

In [41]:
def get_accuracy():
    for i in [1,2,3,4,5,6,7,8,9,10]: 
        t = []
        TRAIN_C_0_new=np.zeros(TRAIN_C_0.shape)
        for i in range(TRAIN_C_0.shape[0]):
            TRAIN_C_0_new[i][np.argmax(TRAIN_C_0[i])]=1
        from scipy import sparse
        TRAIN_Lc=TRAIN_C_0_new.T@TRAIN_L@TRAIN_C_0_new
        TRAIN_Wc=(-1*TRAIN_Lc)*(1-np.eye(TRAIN_Lc.shape[0]))
        TRAIN_Wc[TRAIN_Wc<0.1]=0
        TRAIN_Wc=sparse.csr_matrix(TRAIN_Wc)
        TRAIN_Wc = TRAIN_Wc.tocoo()
        TRAIN_row = torch.from_numpy(TRAIN_Wc.row).to(torch.long)
        TRAIN_col = torch.from_numpy(TRAIN_Wc.col).to(torch.long)

        TRAIN_edge_index_coarsen2 = torch.stack([TRAIN_row, TRAIN_col], dim=0)
        TRAIN_edge_weight = torch.from_numpy(TRAIN_Wc.data)
        def one_hot(x, class_count):
            return torch.eye(class_count)[x, :]

        device = torch.device('cpu')
        TRAIN_Y = TRAIN_labels
        TRAIN_Y = one_hot(TRAIN_Y,NO_OF_CLASSES)
        TRAIN_P=np.linalg.pinv(TRAIN_C_0_new)
        TRAIN_labels_coarse = torch.argmax(torch.sparse.mm(torch.Tensor(TRAIN_P).double() , TRAIN_Y.double()).double() , 1)


        TRAIN_Wc=TRAIN_Wc.toarray()
        TRAIN_adjtemp = torch.tensor(TRAIN_Wc)
        TRAIN_edge_list_temp = dense_to_sparse(TRAIN_adjtemp)[0]
        TRAIN_number_of_edges = TRAIN_edge_list_temp.shape[1]
        n = TRAIN_labels_coarse.shape[0]
        TRAIN_sparsity = 2*TRAIN_number_of_edges/(n*(n-1))

        TRAIN_C2=np.linalg.pinv(TRAIN_C_0_new)
        model=Net().to(device)
        device = torch.device('cpu')
        lr=0.01
        decay=0.0001
        try:
            TRAIN_X=np.array(TRAIN_features.todense())
        except:
            TRAIN_X = np.array(TRAIN_features)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=decay)
        TRAIN_x=sample(range(0, int(TRAIN_K)), TRAIN_K)
        TRAIN_Xt=TRAIN_P@TRAIN_X
        def train():
            model.train()
            optimizer.zero_grad()
            out = model(torch.Tensor(TRAIN_Xt).to(device),TRAIN_edge_index_coarsen2)
            loss = F.nll_loss(out[TRAIN_x], TRAIN_labels_coarse[TRAIN_x])
            loss.backward()
            optimizer.step()
            return loss
        def test():
            model.eval()
            out = model(TEST_X, TEST_edge_index)
            pred = out.argmax(dim=1)  # Use the class with highest probability.
            NO_OF_NODES = test_global_num
            zz=sample(range(0, int(test_global_num)), NO_OF_NODES)
            test_correct = pred[zz] == TEST_Y[zz]  # Check against ground-truth labels.
            test_acc = int(test_correct.sum()) / NO_OF_NODES # Derive ratio of correct predictions.
            return test_acc
        def val():
            model.eval()
            out = model(VAL_X, VAL_edge_index)
            pred = out.argmax(dim=1)  # Use the class with highest probability.
            NO_OF_NODES = VAL_X.shape[0]
            zz=sample(range(0,int(val_global_num)), val_global_num)
            test_correct = pred[zz] == TEST_Y[zz]  # Check against ground-truth labels.
            val_acc = int(test_correct.sum()) / val_global_num # Derive ratio of correct predictions.
            return val_acc

        now1 = datetime.now()
        losses=[]
        for epoch in range(200):
            loss=train()
            losses.append(loss)

        now2 = datetime.now()        
        TRAIN_pred=model(torch.Tensor(TRAIN_Xt).to(device),TRAIN_edge_index_coarsen2).argmax(dim=1)        
        def train_accuracy():
            model.eval()
            correct = (TRAIN_pred[TRAIN_x] == TRAIN_labels_coarse[TRAIN_x]).sum()
            acc = int(correct) /len(TRAIN_x)
            return acc
        t+=[(now2-now1).total_seconds()]
        NO_OF_NODES = TRAIN_X.shape[0]
        zz=sample(range(0, int(NO_OF_NODES)), NO_OF_NODES)
        TRAIN_Wc=sparse.csr_matrix(TRAIN_adj)
        TRAIN_Wc = TRAIN_Wc.tocoo()
        TRAIN_row = torch.from_numpy(TRAIN_Wc.row).to(torch.long)
        TRAIN_col = torch.from_numpy(TRAIN_Wc.col).to(torch.long)
        TRAIN_edge_index_coarsen = torch.stack([TRAIN_row, TRAIN_col], dim=0)
        TRAIN_edge_weight = torch.from_numpy(TRAIN_Wc.data)
        pred=model(torch.Tensor(TRAIN_X),TRAIN_edge_index_coarsen).argmax(dim=1)
        pred=np.array(pred)
        correct =(pred[zz]==TRAIN_labels[zz]).sum()
        acc = int(correct) /NO_OF_NODES
        test_acc = test()
        val_acc = val()
        print(f'Train Accuracy: {acc:.4f}')
        print(f'Test Accuracy: {test_acc:.4f}')
        print(f'Validtion Accuracy: {val_acc:.4f}')
        return acc
        
    
    

In [42]:
get_accuracy()

Train Accuracy: 0.7306
Test Accuracy: 0.5852
Validtion Accuracy: 0.1519


0.7306273062730627

In [43]:
NO_OF_NODES = TEST_X.shape[0]
START = train_global_num
zz=sample(range(START, START + int(test_global_num)), NO_OF_NODES)

In [44]:
test_global_num

270

In [45]:
TEST_X.shape

torch.Size([270, 1433])

In [46]:
len(zz)

270